In [1]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
from sympy.utilities.lambdify import lambdify
from sympy import Matrix
from tkinter import *
import time

#  initialize
_wid = _hei = 700  # window's width and height
wid = hei = 500  # canvas's width and height

# load adjacency and multi-dimensional space
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64)
edge_num = len(edge)
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
node_num, high_dim = P.shape

dim = 3

def genE():
    L = np.sqrt(np.genfromtxt('csv/eigVals.csv', delimiter=",")[0:high_dim])
    base = np.zeros(high_dim * dim).reshape(dim, high_dim)
    for i in range(high_dim): base[i % dim][i] = 1
    E = base*L
    return E

E = genE()

Xs = np.zeros(node_num)
Ys = np.zeros(node_num)
Zs = np.zeros(node_num)
Xs_scaled = np.zeros(node_num)
Ys_scaled = np.zeros(node_num)
Zs_scaled = np.zeros(node_num)
boundingX = 0
boundingY = 0

def scale(pnt,bool):
	if(bool): return wid*(pnt + boundingX/2)/boundingX + (_wid - wid)/2
	else: return (hei-100)*(boundingY/2 - pnt)/boundingY + (_hei - hei)/2

def unscale(pnt,bool):
	if (bool): return boundingX * ((pnt - (_wid - wid)/2) - wid / 2) / wid
	else: return boundingY * ((pnt - (_hei - hei)/2) - (hei - 100) / 2) / (100 - hei)

def update_points():
	for i in range(node_num):
		global Xs, Ys, Zs, boundingX, boundingY
		p0 = P[i, 0:high_dim]
		Xs[i] = np.dot(p0, E[0]) ; Ys[i] = np.dot(p0, E[1]) ; Zs[i] = np.dot(p0, E[2])
	boundingX = max([np.amax(Xs), np.amin(Xs)]) * 2
	boundingY = max([np.amax(Ys), np.amin(Ys)]) * 2
	for i in range(node_num):
		Xs_scaled[i] = scale(Xs[i], True);Ys_scaled[i] = scale(Ys[i], False)

update_points()

print("init: ready")

init: ready


In [6]:
d = high_dim
identifier = "14"
# sympy
a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2 = sp.symbols(
	'a1 b1 c1 d1 a2 b2 c2 d2 a3 b3 c3 d3 t1 s1 u1 t2 s2 u2')  # variables
x2_s, y2_s, z2_s = sp.symbols('x2_s y2_s z2_s')  # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
E3 = sp.MatrixSymbol('E3', d, 1)
_var = (x2_s, y2_s, z2_s, P_i, E1, E2, E3, a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2)

E0 = sp.Matrix(P_i - x2_s * E1 - y2_s * E2 - z2_s * E3)
E0 = sp.simplify(E0 / sp.Matrix.norm(E0))

_E1 = sp.Matrix(a1 * E1 + b1 * E2 + c1 * E3 + d1 * E0)
_E2 = sp.Matrix(a2 * E1 + b2 * E2 + c2 * E3 + d2 * E0)
_E3 = sp.Matrix(a3 * E1 + b3 * E2 + c3 * E3 + d3 * E0)
R1 = sp.Matrix(t1 * E1 + s1 * E2 + u1 * E3)
R2 = sp.Matrix(t2 * E1 + s2 * E2 + u2 * E3)

_f = Matrix([
	_E1.dot(_E1) - 1,
	_E2.dot(_E2) - 1,
	_E3.dot(_E3) - 1,
	_E1.dot(_E2),
	_E2.dot(_E3),
	_E3.dot(_E1),
	R1.dot(R1) - 1,
	R2.dot(R2) - 1,
	R1.dot(R2),
	_E1.dot(R1) - sp.Matrix(E1).dot(R1),
	_E2.dot(R1) - sp.Matrix(E2).dot(R1),
	_E3.dot(R1) - sp.Matrix(E3).dot(R1),
	_E1.dot(R2) - sp.Matrix(E1).dot(R2),
	_E2.dot(R2) - sp.Matrix(E2).dot(R2),
	_E3.dot(R2) - sp.Matrix(E3).dot(R2),
	sp.Matrix(P_i).dot(_E1) - x2_s,
	sp.Matrix(P_i).dot(_E2) - y2_s,
	sp.Matrix(P_i).dot(_E3) - z2_s
])

In [7]:
_func = sp.Matrix.norm(_f)
print("1")
_lam_f = lambdify(_var, _func, 'numpy')
print("2")
def _lam(x2, y2, z2, p, e_1, e_2, e_3):
	return lambda a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2: \
		_lam_f(x2, y2, z2, p, e_1, e_2, e_3, a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2)
print("3")
ons = np.ones(18)
_arr = np.array(ons)

_f2 = _lam(0, 0, 0, P[int(identifier)].reshape(d, 1), E[0].reshape(d,1), E[1].reshape(d,1), E[2].reshape(d,1))
print("4")
def _g(args): return _f2(*args)

res = opt.minimize(_g, _arr, method='L-BFGS-B',options={'ftol':1e-3})
print(res)
e1 = res.x[0] * temp1 + res.x[1] * temp2 + res.x[2] * temp3 + res.x[3] * P[int(identifier)].reshape(d, 1)
e2 = res.x[4] * temp1 + res.x[5] * temp2 + res.x[6] * temp3 + res.x[7] * P[int(identifier)].reshape(d, 1)
e3 = res.x[8] * temp1 + res.x[9] * temp2 + res.x[10] * temp3 + res.x[11] * P[int(identifier)].reshape(d, 1)

1
2
3
4
      fun: 1.1857463743819303
 hess_inv: <18x18 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0524488 ,  0.00854441, -0.03702183,  0.0323634 ,  0.03993095,
        0.11347163, -0.00176736, -0.0330084 ,  0.06583691,  0.00897091,
       -0.02049858,  0.02634357, -0.10904539,  0.16686932,  0.01518758,
        0.05010554, -0.01212099,  0.01619476])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 589
      nit: 27
   status: 0
  success: True
        x: array([ 0.26542961,  0.04337758, -0.13456025,  0.81635749,  0.2980174 ,
       -0.00831947, -0.02402673,  0.98938367,  0.14780906, -0.07477988,
        0.86923879,  0.7234823 , -0.01233021, -0.00601502,  0.65994409,
       -0.01176361, -0.00665626,  0.65994814])


NameError: name 'temp1' is not defined

In [22]:
identifier = "14"

# sympy
a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u = sp.symbols('a1 b1 c1 d1 a2 b2 c2 d2 a3 b3 c3 d3 t s u')   # variables
x2_s,y2_s,z2_s = sp.symbols('x2_s y2_s z2_s')  # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
E3 = sp.MatrixSymbol('E3', d, 1)
_var = (x2_s,y2_s,z2_s,P_i,E1,E2,E3,a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u)

_E1 = sp.Matrix(a1*E1 + b1*E2 + c1*E3 + d1*P_i)
_E2 = sp.Matrix(a2*E1 + b2*E2 + c2*E3 + d2*P_i)
_E3 = sp.Matrix(a3*E1 + b3*E2 + c3*E3 + d3*P_i)
R = sp.Matrix(t*E1 + s*E2 + u*E3)

_f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E3.dot(_E3) - 1,
		_E1.dot(_E2),
		_E2.dot(_E3),
		_E3.dot(_E1),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		_E3.dot(R) - sp.Matrix(E3).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s,
		sp.Matrix(P_i).dot(_E3) - z2_s
		])

_func = sp.Matrix.norm(_f)

_lam_f = lambdify(_var, _func, 'numpy')

def _lam(x2,y2,z2,p,e_1,e_2,e_3):
    return lambda a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u: \
        _lam_f(x2,y2,z2,p,e_1,e_2,e_3,a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3,t,s,u)

    
ons = np.ones(15)
_arr = np.array(ons)

_f2 = _lam(0, 0, 0, P[int(identifier)].reshape(d,1), e1, e2, e3)
def _g(args): return _f2(*args)
res = opt.minimize(_g, _arr, method='L-BFGS-B')
print(res)
e1 = res.x[0] * temp1 + res.x[1] * temp2 + res.x[2] * temp3 + res.x[3] * P[int(identifier)].reshape(d,1)
e2 = res.x[4] * temp1 + res.x[5] * temp2 + res.x[6] * temp3 + res.x[7] * P[int(identifier)].reshape(d,1)
e3 = res.x[8] * temp1 + res.x[9] * temp2 + res.x[10] * temp3 + res.x[11] * P[int(identifier)].reshape(d,1)
update_points()
print(Xs[14],Ys[14],Zs[14])

      fun: 6.9348480821558674e-06
 hess_inv: <15x15 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.47280315,  0.1479708 , -0.46911059, -0.03828469,  0.14711884,
        0.19723734,  0.09504176, -0.4375587 ,  0.436957  ,  0.31229612,
       -0.13059697, -0.14977589, -0.00439372, -0.16080881,  0.20957697])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 2336
      nit: 104
   status: 0
  success: True
        x: array([ 0.96034818,  0.75434511, -0.5929625 ,  0.51933612,  0.23695358,
       -0.02537001,  1.07883479, -0.26572628, -0.22593551,  1.85861003,
       -0.65209731,  0.96837802,  0.77793402, -0.3605194 , -0.51463019])
